# 0. Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

Missing colon in file '/Users/jamesli/.matplotlib/matplotlibrc', line 1 ('TkAgg')


In [2]:
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object

In [3]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

# 1. Rerefence genome data preparation
## 1.1. Check reference genome installation

Celloracle uses genomepy to get DNA sequence data. Before starting celloracle analysis, we need to make sure that the reference genome is correctly installed in your computational environment. If not, please install reference genome.

In [4]:
# PLEASE make sure that you are setting correct ref genome.
ref_genome = "mm10"

genome_installation = ma.is_genome_installed(ref_genome=ref_genome)
print(ref_genome, "installation: ", genome_installation)

mm10 installation:  True


In [5]:
#import genomepy
#genomepy.install_genome("mm10", "UCSC")

## 1.2. Install reference genome (if refgenome is not installed)

In [6]:
if not genome_installation:
    import genomepy
    genomepy.install_genome(ref_genome, "UCSC")
else:
    print(ref_genome, "is installed.")

mm10 is installed.


# 2. Load data

## 2.1. Load processed peak data 
The peak2gene link pairs identified by ArchR was processed with "peak2gene_cellOracle.R" and saved as a Parquet file.

In [8]:
# Load annotated peak data.
peaks = pd.read_parquet("peak_file.parquet")
peaks.head()

,peak_id,gene_short_name
0,chr11_70203890_70204390,Acap1
1,chr11_70203890_70204390,Alox12
2,chr11_83284456_83284956,Slfn5
3,chr3_20027269_20027769,Trim55
4,chr11_83649364_83649864,Ccl4


## 2.1. Check data

In [9]:
# Define function for quality check
def decompose_chrstr(peak_str):
    """
    Args:
        peak_str (str): peak_str. e.g. 'chr1_3094484_3095479'
        
    Returns:
        tuple: chromosome name, start position, end position
    """
    
    *chr_, start, end = peak_str.split("_")
    chr_ = "_".join(chr_)
    return chr_, start, end

from genomepy import Genome

def check_peak_foamat(peaks_df, ref_genome):
    """
    Check peak fomat. 
     (1) Check chromosome name. 
     (2) Check peak size (length) and remove sort DNAs (<5bp)
    
    """
    
    df = peaks_df.copy()
    
    n_peaks_before = df.shape[0]
    
    # Decompose peaks and make df
    decomposed = [decompose_chrstr(peak_str) for peak_str in df["peak_id"]]
    df_decomposed = pd.DataFrame(np.array(decomposed))
    df_decomposed.columns = ["chr", "start", "end"]
    df_decomposed["start"] = df_decomposed["start"].astype(np.int)
    df_decomposed["end"] = df_decomposed["end"].astype(np.int)
    
    # Load genome data
    genome_data = Genome(ref_genome)
    all_chr_list = list(genome_data.keys())
    
    
    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])
    
    
    # Filter peaks with invalid chromosome name
    n_threshold = 5
    df = df[(lengths >= n_threshold) & df_decomposed.chr.isin(all_chr_list)]
    
    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])
    
    # Data counting
    n_invalid_length = len(lengths[lengths < n_threshold])
    n_peaks_invalid_chr = n_peaks_before - df_decomposed.chr.isin(all_chr_list).sum()
    n_peaks_after = df.shape[0]
    
    #
    print("Peaks before filtering: ", n_peaks_before)
    print("Peaks with invalid chr_name: ", n_peaks_invalid_chr)
    print("Peaks with invalid length: ", n_invalid_length)
    print("Peaks after filtering: ", n_peaks_after)
    
    return df

In [10]:
peaks = check_peak_foamat(peaks, ref_genome)

Peaks before filtering:  197299
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  197299


## 2.2 Load motif

In [10]:
from gimmemotifs.motif import read_motifs
# Read modified gimme.vertebrate.v5.0 with added Bcl11b motif
path = 'gimme.vertebrate.v5.0m/gimme.vertebrate.v5.0.pfm'
print(path)
motifs = read_motifs(path)
motifs[0:10]

gimme.vertebrate.v5.0m/gimme.vertebrate.v5.0.pfm


[GM.5.0.Sox.0001_AACAAT,
 GM.5.0.Homeodomain.0001_AGCTGTCAnnA,
 GM.5.0.Mixed.0001_snnGGsssGGs,
 GM.5.0.Nuclear_receptor.0001_TAwsTrGGTCAsTrGGTCA,
 GM.5.0.Mixed.0002_GCTAATTA,
 GM.5.0.Nuclear_receptor.0002_wnyrCTTCCGGGkC,
 GM.5.0.bHLH.0001_ACGTG,
 GM.5.0.Myb_SANT.0001_rrCCGTTAAACnGyy,
 GM.5.0.C2H2_ZF.0001_GCGkGGGCGG,
 GM.5.0.GATA.0001_TTATCTsnnnnnnnCA]

# 3. Instantiate TFinfo object and search for TF binding motifs
The motif analysis module has a custom class; TFinfo. The TFinfo object converts a peak data into a DNA sequences and scans the DNA sequences searching for TF binding motifs. Then, the results of motif scan will be filtered and converted into either a python dictionary or a depending on your preference. This TF information is necessary for GRN inference.

## 3.1. Instantiate TFinfo object

In [11]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks, # peak info calculated from ATAC-seq data
                       ref_genome=ref_genome) 

## 3.2. Motif scan


!!You can set TF binding motif information as an argument: 

tfi.scan(motifs=motifs)

If you don't set motifs or set None, default motifs will be loaded automatically.

- For mouse and human, "gimme.vertebrate.v5.0." will be used as a default motifs. 
- For another species, a species specific TF binding motif data extracted from CisBP ver2.0 will be used.



In [12]:
%%time
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02, 
            motifs=None,  # If you enter None, default motifs will be loaded.
            verbose=True)

# Save tfinfo object
tfi.to_hdf5(file_path="cCRE.celloracle.tfinfo")

No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please go https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 

Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 

Convert peak info into DNA sequences ... 

Scanning motifs ... It may take several hours if you proccess many peaks. 




CPU times: user 1h 37min 16s, sys: 2min 8s, total: 1h 39min 24s
Wall time: 1h 39min 57s


In [13]:
# Check motif scan results
tfi.scanned_df.head()

,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100004434_100004934,GM.5.0.Sox.0002,"SOX9, SRY, Sry",Sox9,8.502153,465,1
1,chr10_100004434_100004934,GM.5.0.bZIP.0004,"CEBPG, Chop, Ddit3, Cebpg, Atf4, ATF4","Atf5, MYC, ATF3, Atf4, ATF4",8.044841,459,-1
2,chr10_100004434_100004934,GM.5.0.GATA.0002,"GATA2, GATA3",Gata2,8.484434,378,1
3,chr10_100004434_100004934,GM.5.0.Ets.0004,,"ZBTB7A, ELF1, GABPA, ELK4",3.959890,266,-1
4,chr10_100004434_100004934,GM.5.0.E2F.0001,"E2F4, E2F2, E2F3, E2F1","E2f1, E2F4, E2f5, E2F1, E2f4, E2F2, E2F3",5.097280,108,1


In [14]:
df=tfi.scanned_df
len(df)

17026486

In [17]:
folder='TFinfo_outputs'

In [18]:
# save the entire scanned results
df.to_parquet(os.path.join(folder,"tfi_scanned.parquet"))

# 4. Filtering motifs
Out of 17026486, 3118211 passed the filtering (18.31%)

In [19]:
# Reset filtering 
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10.5)

# Do post filtering process. Convert results into several file format.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

Filtering finished: 17026486 -> 3118211
1. Converting scanned results into one-hot encoded dataframe.



2. Converting results into dictionaries.


We have the score for each sequence and motif_id pair.
In the next step we will filter the motifs with low score.

# 5. Get Final results

## 5.1. Get resutls as a dictionary

In [20]:
td = tfi.to_dictionary(dictionary_type="targetgene2TFs")


## 5.2. Get results as a dataframe

In [21]:
df = tfi.to_dataframe()
df.head()

,peak_id,gene_short_name,9430076c15rik,Ac002126.6,Ac012531.1,Ac226150.2,Afp,Ahr,Ahrr,Aire,...,Znf784,Znf8,Znf816,Znf85,Zscan10,Zscan16,Zscan22,Zscan26,Zscan31,Zscan4
0,chr10_100004434_100004934,Kitl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chr10_100015536_100016036,Cep290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,chr10_100015536_100016036,Kitl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,chr10_100015536_100016036,Tmtc3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,chr10_100016814_100017314,Kitl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 6. Save TFinfo as dictionary or dataframe
We'll use this information when making the GRNs. Save the results.

In [22]:
folder = "TFinfo_outputs"
os.makedirs(folder, exist_ok=True)

# save TFinfo as a dictionary
td = tfi.to_dictionary(dictionary_type="targetgene2TFs")
save_as_pickled_object(td, os.path.join(folder, "TFinfo_targetgene2TFs.pickled"))

# save TFinfo as a dataframe
df = tfi.to_dataframe()
df.to_parquet(os.path.join(folder, "TFinfo_dataframe.parquet"))